# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy import stats as st

In [2]:

try:
    geo0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    geo1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    geo2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
for data in geo0, geo1, geo2:
    print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [5]:
for data in geo0, geo1, geo2:
    print(data.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [6]:
for data in geo0, geo1, geo2:
    print(data['id'].duplicated().sum())

10
4
4


In [7]:
for data in geo0, geo1, geo2:
    drop = data['id'].drop_duplicates()
    print(drop)


0        txEyH
1        2acmU
2        409Wp
3        iJLyR
4        Xdl7t
         ...  
99995    DLsed
99996    QKivN
99997    3rnvd
99998    7kl59
99999    1CWhH
Name: id, Length: 99990, dtype: object
0        kBEdx
1        62mP7
2        vyE1P
3        KcrkZ
4        AHL4O
         ...  
99995    QywKC
99996    ptvty
99997    09gWa
99998    rqwUm
99999    relB0
Name: id, Length: 99996, dtype: object
0        fwXo0
1        WJtFt
2        ovLUW
3        q6cA6
4        WPMUX
         ...  
99995    4GxBu
99996    YKFjq
99997    tKPY3
99998    nmxp2
99999    V9kWn
Name: id, Length: 99996, dtype: object


прочитали даниие 3 датасетов из 3 регионов нашли и удалили дубликаты в столбце id так как повтор данных выдал бы нам не коректные данные.

## Обучение и проверка модели

проверка geo0

In [8]:
geo0_train, geo0_valid = train_test_split(geo0, test_size=0.25, random_state=12345)
features_train_geo0 = geo0_train.drop(['product', 'id'], axis=1)
target_train_geo0 = geo0_train['product']

features_valid_geo0 = geo0_valid.drop(['product', 'id'], axis=1)
target_valid_geo0 = geo0_valid['product'] 

print(features_train_geo0.shape)
print(features_valid_geo0.shape)

model = LinearRegression()
model.fit(features_train_geo0, target_train_geo0)
predicted_valid_geo0 = model.predict(features_valid_geo0)
mse = mean_squared_error(target_valid_geo0, predicted_valid_geo0)


print("Linear Regression")
print("MSE =", mse)
print("RMSE =", mse ** 0.5)
print("Mean =", predicted_valid_geo0.mean())

(75000, 3)
(25000, 3)
Linear Regression
MSE = 1412.2129364399243
RMSE = 37.5794217150813
Mean = 92.59256778438035


проверка geo1

In [9]:
geo1_train, geo1_valid = train_test_split(geo1, test_size=0.25, random_state=12345)
features_train_geo1 = geo1_train.drop(['product', 'id'], axis=1)
target_train_geo1 = geo1_train['product']

features_valid_geo1 = geo1_valid.drop(['product', 'id'], axis=1)
target_valid_geo1 = geo1_valid['product'] 

print(features_train_geo1.shape)
print(features_valid_geo1.shape)

model = LinearRegression()
model.fit(features_train_geo1, target_train_geo1)
predicted_valid_geo1 = model.predict(features_valid_geo1)
mse = mean_squared_error(target_valid_geo1, predicted_valid_geo1)


print("Linear Regression")
print("MSE =", mse)
print("RMSE =", mse ** 0.5)
print("Mean =", predicted_valid_geo1.mean())

(75000, 3)
(25000, 3)
Linear Regression
MSE = 0.7976263360391157
RMSE = 0.893099286775617
Mean = 68.728546895446


проверка geo2

In [10]:
geo2_train, geo2_valid = train_test_split(geo2, test_size=0.25, random_state=12345)
features_train_geo2 = geo2_train.drop(['product', 'id'], axis=1)
target_train_geo2 = geo2_train['product']

features_valid_geo2 = geo1_valid.drop(['product', 'id'], axis=1)
target_valid_geo2 = geo1_valid['product'] 

print(features_train_geo2.shape)
print(features_valid_geo2.shape)

model = LinearRegression()
model.fit(features_train_geo2, target_train_geo2)
predicted_valid_geo2 = model.predict(features_valid_geo2)
mse = mean_squared_error(target_valid_geo2, predicted_valid_geo2)


print("Linear Regression")
print("MSE =", mse)
print("RMSE =", mse ** 0.5)
print("Mean =", predicted_valid_geo2.mean())

(75000, 3)
(25000, 3)
Linear Regression
MSE = 2020.1343101336374
RMSE = 44.945904264277935
Mean = 95.23295059251816


## Подготовка к расчёту прибыли

In [11]:
tochki = 500#all_points
nuj_tochki=200
viborki = 1000#bootstrap_samples 
bud_obsh = 10000000000# total_budget
bud_1= bud_obsh/nuj_tochki #
bars = 450000 #cost_of_one
bar = 450 #barell_profit
granica_riska= 0.025 #risk_treshold

In [12]:
kol_bar=bud_1/bars # нужное количество барелей что бы окупилась разработка
kol_bar

111.11111111111111

In [13]:
print('предсказанный объём сырья со средним запасом в регионе geo0:',predicted_valid_geo0.mean())
print('предсказанный объём сырья со средним запасом в регионе geo1:',predicted_valid_geo1.mean())
print('предсказанный объём сырья со средним запасом в регионе geo2:',predicted_valid_geo2.mean())

предсказанный объём сырья со средним запасом в регионе geo0: 92.59256778438035
предсказанный объём сырья со средним запасом в регионе geo1: 68.728546895446
предсказанный объём сырья со средним запасом в регионе geo2: 95.23295059251816


Нужно 111 тыc баррелей на разработку одной сквжины. Получается, не один регион не окупил себя.

In [14]:
def revenue(target, predictions):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    predictions_s = predictions.sort_values(ascending=False)[:nuj_tochki].index
    selected = target[predictions_s]
    revenue = selected.sum() * bars - bud_obsh
    return round(revenue, 2)

## Расчёт прибыли и рисков 

In [15]:
revenue(target_valid_geo0, predicted_valid_geo0)

3320826043.14

In [16]:
revenue(target_valid_geo1, predicted_valid_geo1)

2415086696.68

In [17]:
revenue(target_valid_geo2, predicted_valid_geo2)

2143273425.82

In [18]:
def bootstrap(target, probabilities, iterations, sample_size):
    state = np.random.RandomState(12345)
    values = []
    for i in range(iterations):
        target_subsample = target.sample(n=sample_size, random_state=state, replace=True)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample))

    values = pd.Series(values)
    mean = values.mean()
    risk = (values < 0).mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    quantile_interval = (lower, upper)
    return round(mean,2), confidence_interval, round(risk*100, 2), quantile_interval


In [19]:
geo0_mean, geo0_interval, geo0_risk, geo0_quantile = bootstrap(target_valid_geo0.reset_index(drop=True), predicted_valid_geo0, 1000, 500)
print('Регион geo0 средняя выручка = ',  geo0_mean)
print('Регион geo0 95 %ый доверительный интервал по стьюденту = ', geo0_interval)
print('Регион geo0 95 %ый доверительный интервал по квантилям = ',  geo0_quantile)
print('Регион geo0 Риск убытков: ',geo0_risk,'%')


Регион geo0 средняя выручка =  396164984.8
Регион geo0 95 %ый доверительный интервал по стьюденту =  (379620315.14788723, 412709654.4566728)
Регион geo0 95 %ый доверительный интервал по квантилям =  (-111215545.89, 909766941.5539998)
Регион geo0 Риск убытков:  6.9 %


In [20]:
geo1_mean, geo1_interval, geo1_risk, geo1_quantile = bootstrap(target_valid_geo1.reset_index(drop=True), predicted_valid_geo1, 1000, 500)
print('Регион geo1 средняя выручка = ', geo1_mean)
print('Регион geo1 95 %ый доверительный интервал по стьюденту = ', geo1_interval)
print('Регион geo1 95 %ый доверительный интервал по квантилям = ', geo1_quantile)
print('Регион geo1 Риск убытков: ',geo1_risk,'%')


Регион geo1 средняя выручка =  456045105.79
Регион geo1 95 %ый доверительный интервал по стьюденту =  (443147248.6638929, 468942962.909427)
Регион geo1 95 %ый доверительный интервал по квантилям =  (33820509.397750005, 852289453.864)
Регион geo1 Риск убытков:  1.5 %


In [21]:
geo2_mean, geo2_interval, geo2_risk, geo2_quantile = bootstrap(target_valid_geo2.reset_index(drop=True), predicted_valid_geo2, 1000, 500)
print('Регион geo2 средняя выручка = ', geo2_mean)
print('Регион geo2 95 %ый доверительный интервал по стьюденту = ', geo2_interval)
print('Регион geo2 95 %ый доверительный интервал по квантилям = ', geo2_quantile)
print('Регион geo2 Риск убытков: ',geo2_risk,'%')

Регион geo2 средняя выручка =  421122822.87
Регион geo2 95 %ый доверительный интервал по стьюденту =  (407967108.7912553, 434278536.93912476)
Регион geo2 95 %ый доверительный интервал по квантилям =  (-1128021.85, 833798998.46625)
Регион geo2 Риск убытков:  2.6 %


# Вывод:

прочитали даниие 3 датасетов из 3 регионов нашли и удалили дубликаты в столбце id так как повтор данных выдал бы нам не коректные данные.
обучили и предсказали данные методом линейной регресии все 3 региона.
В результате видим, что самые большие средние запасы сырья находятся в регионе 3. При этом значение rmse довольно высокое. Большое значение rmse региона говорит о большом разбросе ошибок, эти значения самые высокие для geo 0 и 2.
нашли объём сырья со средним запасом в регионе
нашли нужное количество барелей что бы окупилась разработка
Согласно прогнозам сделанным с помощью техники Bootstrap, наиболее прибыльным будет разработка месторождений в регионе geo1. В данном регионе самые высокие показатели средней выручки и 95%-ного доверительного интервала. Также в регионе geo1 самый низкий риск убытков.